# 零样本图像识别方法v1.0（属性向量方法）

## 初步数据处理

### attributes_per_class.txt 类别级属性标注标签ID(230)对应属性ID标注(30)

In [10]:
f = open(r'DatasetB_20180919/attributes_per_class.txt')
data_per_class = f.readlines()
data = []
f.close()
for line in data_per_class:

    data.append(line.split())

**提取标签id**

In [11]:
IDs= []
for ID_index in data:
    IDs.append(ID_index[0])

### 对lable做一次清洗

**attribute_list属性类合计head**

In [12]:
head = list(['lable',

    'animal',

'transportation',

'clothes',

'plant',

'tableware',

'device',

'black',

'white',

'blue',

'brown',

'orange',

'red',

'green',

'yellow',

'has_feathers',

'has_four_legs',

'has_two_legs',

'has_two_arms',

'for_entertainment',

'for_business',

'for_communication',

'for_family',

'for_office use',

'for_personal',

'gorgeous',

'simple',

'elegant',

'cute',

'pure',

'naive'])

**pandas数据处理**

**将属性分成了五类 类别、颜色（未知原因暂时忽略）、形态、 用途、性格（此类别主观性太强暂且不用**、

**改进思路：将标签的每一行做清洗只留下一类为1**

In [13]:
#对lable做一次清洗
import numpy as np

import pandas as pd

data = pd.DataFrame(data,columns= head)
#将标签ID与30位属性向量对应起来，生成表格形式的数据结构

# data = data.set_index('lable')

 

data_1 = data.loc[:,'animal':'device']

data_2 = data.loc[:,'black':'yellow']

data_3 = data.loc[:,'has_feathers':'has_two_arms']

data_4 = data.loc[:,'for_entertainment':'for_personal']

data_5 = data.loc[:,'gorgeous':'naive']


 
#将属性做清理，此处可作为改进的地方
 
# data_1 = (data_1 >= '1') & 1
data_1 = data_1.astype('float')
m, n = data_1.shape
data_1_sum = data_1.sum(axis = 1)
for i in range(m):
    data_1.iloc[i] = data_1.iloc[i] / data_1_sum[i]
data_1 = data_1.fillna(0)
data_1['non_1'] = 1 - data_1.sum(axis = 1)
# 此处在data_中加入了一行

 

# #颜色属性不太好用（比如car\pan\cost），这里就不做复杂处理，模型训练时暂不用

# data_2 = (data_2 > '0.7') & 1
data_2 = data_2.astype('float')
m, n = data_2.shape
# data_2 = np.exp(data_2)
data_2_sum = data_2.sum(axis = 1)
for i in range(m):
    data_2.iloc[i] = data_2.iloc[i] / data_2_sum[i]
data_2 = data_2.fillna(0)
data_2['non_2'] =1 - data_2.sum(axis = 1)
# data_2 = data_2.astype('str')
# data_2.sum(axis = 1)

 

# data_3 = (data_3 >= '1') & 1
data_3 = data_3.astype('float')
m, n = data_3.shape
# data_3= np.exp(data_3)
data_3_sum = data_3.sum(axis = 1)
for i in range(m):
    data_3.iloc[i] = data_3.iloc[i] / data_3_sum[i]
data_3 = data_3.fillna(0)
data_3['non_3'] =1 - data_3.sum(axis = 1)
# data_3 = data_3.astype('str')
# data_3.sum(axis = 1)

 

# data_4 = (data_4 >= '1') & 1
data_4 = data_4.astype('float')
m, n = data_4.shape
# data_4 = np.exp(data_4)
data_4_sum = data_4.sum(axis = 1)
for i in range(m):
    data_4.iloc[i] = data_4.iloc[i] / data_4_sum[i]
data_4 = data_4.fillna(0)
data_4['non_4'] =1 - data_4.sum(axis = 1)
# data_4 = data_4.astype('str')
# data_4.sum(axis = 1)



data_new = pd.DataFrame(data['lable'])

data_new = pd.DataFrame(np.hstack((data_new, data_1, data_2, data_3, data_4)))
#重组形成新的data数据表格并保存成csv格式
data_new.to_csv("test_1.csv", index=None)

print("data saved to test_1.csv.")

data saved to test_1.csv.


### 生成测试集的便签对应向量（因为测试集样本排除训练集中出现的部分）

In [24]:
import numpy as np

import random


data_train = open(r'DatasetB_20180919/train.txt')

data_train = data_train.readlines()

length = len(data_train)

dic_train={}
#此处跟随样本分类种类设定

for i in range(length):

    m,n = data_train[i].split()
    
    if n not in dic_train:
        
        dic_train[n]=1

In [25]:
#对lable做一次清洗
import numpy as np

import pandas as pd

class_wordembedding = data_new
#将标签ID与30位属性向量对应起来，生成表格形式的数据结构
count=0
for i in range(285-count):
#     class_wordembedding.iloc[i-count][0] = label_lists[class_wordembedding.iloc[i-count][0]]
    if class_wordembedding.iloc[i-count][0] in dic_train:
        class_wordembedding.drop(class_wordembedding.index[i-count],inplace=True)
        count =count+1

class_wordembedding.to_csv("test_2.csv", index=None)

### 导入训练集数据

In [ ]:
#导入一些之后需要的参数
from keras.preprocessing import image
import pandas as pd

import numpy as np

import random

data_atten = pd.read_csv(r'test_1.csv')

data_atten = data_atten.set_index('0')
# 通过set_index方法，可以设置单索引和复合索引。

data_train = open(r'DatasetB_20180919/train.txt')

data_train = data_train.readlines()

random.shuffle(data_train)

data_train = data_train[0:10000]
#此处为了加快速度只输入了10000张样本

path = r'DatasetB_20180919/train/'

length = len(data_train)

train_x = np.zeros((length, 64, 64, 3))

train_y = np.zeros((length, 28))
#此处跟随样本分类种类设定

for i in range(length):

    m,n = data_train[i].split()

    img = image.load_img(path + m)

    
    train_x[i] = image.img_to_array(img)/255

    train_y[i] = data_atten.loc[n]

## 模型建立与训练

### 建立模型

In [8]:
#build model

from keras import Model, Sequential

from keras.applications import VGG16, VGG19

from keras.layers import Flatten, Dense,Dropout

from keras.preprocessing import image

from sklearn.model_selection import train_test_split

from keras.optimizers import *

from keras import regularizers


base_model = VGG19(include_top=False, weights=None, input_shape=(64, 64, 3))

#weights=“imagenet”可设置成

classes = {

    'cla' : 6+1,

    'clo' : 8+1, 

    'has' : 4+1,

    'for' : 6+1

  #  'is' : 6 #暂且不用

}
#此处cla类+1的意义不知道是什么，可做删除测试
 

x = base_model.output

x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation='relu')(x)
#全连接层
predictions = [Dense(n, activation='softmax', name=m,kernel_regularizer=regularizers.l2(0.01))(x) for m,n in classes.items()]
# predictions = [Dense(n, activation='softmax', name=m,kernel_regularizer=regularizers.l2(0.01))(x) for m,n in classes.items()]

 

model = Model(inputs=base_model.input, outputs= predictions)

optimizer = SGD(lr=0.01, momentum=0., decay=0., nesterov=False)
#此处可修改优化器，在model.compile中optimizer修正
# optimizer = RMSprop(lr=0.001,rho=0.9,epsilon=1e-06)

model.compile(optimizer=optimizer, loss='mse')

#loss可尝试使用mse
model.summary()


Using TensorFlow backend.
/home/jiangdong/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 64, 64, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 64, 64, 64)   36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 32, 32, 64)   0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

### 模型训练与保存（因为无法直观的从loss中看出样本对应标签的正确率，因此中间保存模型并测试）

In [ ]:
#  ```python
#训练模型
# from keras.callbacks import EarlyStopping

# early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='min')
for epo in range(1,300):  # 迭代次数为100次
    print('epochs:', epo)
    h = model.fit(train_x, {'cla':train_y[:,:7],'clo':train_y[:,7:16], 'has':train_y[:,16:21], 'for':train_y[:,21:28]},epochs=3,validation_split=0.1)
    print('\n第', epo, '个epoch完成\n')
    model.save('/model_' + 'epoch_' + str(epo) + '.h5')
    print(h.history)
    #train_y对应所选择输入利用的标签，
#validation_split预留样本做验的比例，可以设定较小一点
#  ```
 

epochs: 1
Train on 44125 samples, validate on 4903 samples
Epoch 1/3
44125/44125 [==============================] - 109s 2ms/step - loss: 0.0923 - cla_loss: 0.0168 - clo_loss: 0.0328 - has_loss: 0.0104 - for_loss: 0.0294 - val_loss: 0.2618 - val_cla_loss: 0.0956 - val_clo_loss: 0.0411 - val_has_loss: 0.0701 - val_for_loss: 0.0521
Epoch 2/3
44125/44125 [==============================] - 113s 3ms/step - loss: 0.0922 - cla_loss: 0.0166 - clo_loss: 0.0326 - has_loss: 0.0108 - for_loss: 0.0293 - val_loss: 0.2630 - val_cla_loss: 0.0975 - val_clo_loss: 0.0408 - val_has_loss: 0.0704 - val_for_loss: 0.0516
Epoch 3/3
44125/44125 [==============================] - 113s 3ms/step - loss: 0.0899 - cla_loss: 0.0158 - clo_loss: 0.0324 - has_loss: 0.0102 - for_loss: 0.0287 - val_loss: 0.2695 - val_cla_loss: 0.0990 - val_clo_loss: 0.0414 - val_has_loss: 0.0739 - val_for_loss: 0.0524

第 1 个epoch完成

{'val_loss': [0.26177108585652153, 0.26304296463235599, 0.26949555040402678], 'val_cla_loss': [0.0956099731

## 测试与分析

### 导入模型参数直接进行预测（用于在windows上测试导入模型）

In [16]:
import pandas as pd

import numpy as np

from keras.models import load_model
 
model = load_model('0921/model_attribute0921.h5')
#导入目录下的模型文件并作出预测

model.summary()

Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 64, 64, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 64, 64, 64)   36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 32, 32, 64)   0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

## 直接输出结果输出结果

**文件读取并输出模型输出特征向量**

In [27]:
from keras.preprocessing import image

data_test = open(r"DatasetB_20180919/image.txt")

data_test = data_test.readlines()

path_test = r'DatasetB_20180919/test/'

length = len(data_test)

data_test_x = np.zeros((length, 64, 64, 3))

for i in range(length):

    m = data_test[i][:-1]

    img = image.load_img(path_test + m)

    data_test_x[i] = image.img_to_array(img)/255
    
res = model.predict(data_test_x)


**读取可能的标签向量**

In [47]:
data_atten = pd.read_csv(r'test_2.csv')

IDs= []
for i in range(121):
    IDs.append(data_atten.loc[i][0])
data_atten = data_atten.set_index('0')

**遍历便签对比获得每一张图片的输出结果并保存**

In [48]:
#遍历标签ID与对应属性值
data_atten_res = []

dic={}

count = 0

length = len(res[0])


for ID in IDs:

    key = ID
#     print(key)
    
    data_atten_res.append(np.hstack((data_atten.loc[key])))
    
    dic[count] = key

    count = count+1

#将预测输出进行处理，组合相对应的
vec_result = []#np.zores((len(res[0]), 18))

for i in range(len(res[0])):

    vec_result.append(np.hstack((res[0][i], res[1][i], res[2][i],res[3][i])))

dis_list = []

file  = open("0921/result092101.txt",'a+')
# file  = open("retaines_test.txt",'a+')
accury = 0 
for i in range(length):
    dis_list=[]
    m= data_test[i]
    for j in range(len(data_atten_res)):

        vec1 = vec_result[i]
        
        vec2 = data_atten_res[j]

        dist = np.linalg.norm(vec1 - vec2)

        dis_list.append(dist)
    m = data_test[i][:-1]
    print(m)
    print("witch is : "+ dic[dis_list.index(min(dis_list))] + " and min dis = " + str(min(dis_list)))

    file.write(m+'\t'+dic[dis_list.index(min(dis_list))]+'\n')
file.close()

0003ae092034aa69da9782b2a3b4a15a.jpg
witch is : ZJL272 and min dis = 0.1651486951574267
000a5f73edc44c5d734ecf4ec9a183b8.jpg
witch is : ZJL214 and min dis = 1.1727484720591481
000d8bf47339a2fab8a058a286b0cd77.jpg
witch is : ZJL184 and min dis = 0.7237052160842463
0012a4bdd0caae038defb2f634978e42.jpg
witch is : ZJL217 and min dis = 0.9741565902985296
0015fa45b8d32bd2ede711292eb50b67.jpg
witch is : ZJL266 and min dis = 1.2278442204166655
002584a5184358af41e4c3782b5e004f.jpg
witch is : ZJL238 and min dis = 0.6209494097265037
002842af1ee22536208434790b178ceb.jpg
witch is : ZJL166 and min dis = 0.43655889146005583
0030ec40c8fc27daf76a80dc1ad7598d.jpg
witch is : ZJL165 and min dis = 0.8331789236790266
0034898602418b888ca80a5fd6d7e2b2.jpg
witch is : ZJL272 and min dis = 0.7259565043095005
003caeebdf4c0305e003168719d6827a.jpg
witch is : ZJL184 and min dis = 0.46286335514377547
003eb87fca28d23a9594f3cab7f0dcdd.jpg
witch is : ZJL183 and min dis = 0.07554912498744805
0044d368883683296227b61e4c598

0283eeaee2d2f50a2b42b80282ccd95e.jpg
witch is : ZJL184 and min dis = 0.4447235652026089
02871c9d40575111b2c1fe75d76ab775.jpg
witch is : ZJL285 and min dis = 0.4944384216617124
028ce09297f3945d4338751065d53dd0.jpg
witch is : ZJL210 and min dis = 0.23888188452466838
028d6c801d3d2d414daedbe6565400b1.jpg
witch is : ZJL207 and min dis = 0.9547925305465551
029a8a1186dcb24ec0e06a5cadb70241.jpg
witch is : ZJL289 and min dis = 0.24470702791070092
029d2fd25667ba2028b81e455c116a90.jpg
witch is : ZJL253 and min dis = 0.36597119741732814
02a2ee5c2055f3cfe5309299f031d0df.jpg
witch is : ZJL232 and min dis = 0.4714464385041303
02a5f2994a049db5d84cb4fc90c85271.jpg
witch is : ZJL238 and min dis = 0.2345064864971336
02a7a9bf3ea8bb6221750130cce8d28a.jpg
witch is : ZJL160 and min dis = 0.7824279807193313
02a7cc9e914ed7468022a5212b635595.jpg
witch is : ZJL240 and min dis = 1.2534425919966243
02a8614f92cdfedb6c66fc0075148f6a.jpg
witch is : ZJL245 and min dis = 0.32446674089677197
02ada4a2da2f4ca11b51a09a1a32

044aa78828a07c418a7ef30f26cd92b9.jpg
witch is : ZJL166 and min dis = 1.0297603357493341
04582495815cdadf0073d71bd1d12314.jpg
witch is : ZJL230 and min dis = 0.44346305393417107
047aefbea9c0efa4bc85a848d283e2c4.jpg
witch is : ZJL206 and min dis = 1.206059993374426
0488b0a64fc382eee217f5164ba30802.jpg
witch is : ZJL266 and min dis = 1.229325415605723
04a567d8248d707c457d8500f7a95ab8.jpg
witch is : ZJL166 and min dis = 0.8573315236981462
04a7985ae49c98c129675d9fbf01805f.jpg
witch is : ZJL210 and min dis = 0.4119907424506755
04b87f100d48f1e535fb690210d0d17e.jpg
witch is : ZJL175 and min dis = 0.26869729544238874
04c48b275578c1037d623fb3de6e757c.jpg
witch is : ZJL259 and min dis = 0.4853010556389757
04d0cb092c4fc7e385a2c18e2c29c180.jpg
witch is : ZJL186 and min dis = 0.46860445897126723
04d96dcd76a2e78086ad99726047935c.jpg
witch is : ZJL259 and min dis = 0.7682112072923539
04dd221c0720baaa65a93be0427d57db.jpg
witch is : ZJL257 and min dis = 0.3902084652419867
04f05e45d8c437750b637a4a572d1ae

08c320788b82346afd67ed48dbcfe76d.jpg
witch is : ZJL259 and min dis = 0.7034651407465262
08c591e9c6c8867ef3859e15fb7ef9ed.jpg
witch is : ZJL208 and min dis = 0.9354920124929041
08de10ffa31818f15db03cf8189b84b3.jpg
witch is : ZJL294 and min dis = 0.6199528762472164
08de922fd584ff7f4c17c73f12b69bd1.jpg
witch is : ZJL216 and min dis = 1.0008643711071952
08e8bc55485e564b463c543774a08b04.jpg
witch is : ZJL175 and min dis = 0.4392050058562995
08f983e9fe1a42441b6390b4b844c86f.jpg
witch is : ZJL185 and min dis = 0.8535327508409397
0900dc30220f1edf671170032123af1f.jpg
witch is : ZJL166 and min dis = 0.2054173764839286
0905bf707a873cbaa25b2d456bd2ca34.jpg
witch is : ZJL208 and min dis = 0.580352346645792
090781ecf1778454856944f4c8e3e8fd.jpg
witch is : ZJL253 and min dis = 0.37225031139844866
090fb12e93b6b6a59fd804c144d3ab3a.jpg
witch is : ZJL220 and min dis = 0.26521758247665483
0917742ceac687fcd0b45a45f6323e15.jpg
witch is : ZJL253 and min dis = 1.1118273332232014
0918cb5a8f27ce75545bef698799b82

0c6051e569fb23785f6913135ca18d56.jpg
witch is : ZJL236 and min dis = 1.103124126314532
0c639644df7c3dbf76cbb73a2bbd4595.jpg
witch is : ZJL217 and min dis = 1.121082527534237
0c6ad693941c7c0a4b88eb2eac7b383e.jpg
witch is : ZJL216 and min dis = 1.1791377366018694
0c6bac32166d89f331c21f21deece435.jpg
witch is : ZJL236 and min dis = 0.12731504648033917
0c739fcf22ca9d419ce3e40ce5b5bb01.jpg
witch is : ZJL164 and min dis = 0.588037772239326
0c73eb8deda07a4442f983d89e7a0e5c.jpg
witch is : ZJL259 and min dis = 1.1201559672377557
0c7e0ae10f2daf16b575db1027128cf7.jpg
witch is : ZJL236 and min dis = 0.28003735984883854
0c812e2db1c30386231df3f77da9008e.jpg
witch is : ZJL202 and min dis = 0.5675750381068313
0c81721f102ed97319dca26d9d89a0c7.jpg
witch is : ZJL266 and min dis = 1.3206475118902385
0c838b1d88f8a07ad27f2d22c0aa587d.jpg
witch is : ZJL175 and min dis = 0.28211956475522987
0c86ff826e66bfdcb34724037aa17fd4.jpg
witch is : ZJL165 and min dis = 0.27837968166505606
0c8fe755375c36ee91e51151bba432d

...

f3f0a61168427e08eb1254799fd458a1.jpg
witch is : ZJL259 and min dis = 0.5508704332393746
f4007fa6cfedccdccdbdf3ed8fa109bc.jpg
witch is : ZJL271 and min dis = 0.4097983289636154
f4039b1a637b61211fe3589c5e4ce468.jpg
witch is : ZJL183 and min dis = 0.5293104589437481
f4087d2abb54b13f16e678f22a7b6843.jpg
witch is : ZJL253 and min dis = 0.4082481379974965
f40e4927ad2f58e3907c441438df2328.jpg
witch is : ZJL253 and min dis = 0.9574698992141867
f40fc7b1031fdb1233fa580cd4d89bcc.jpg
witch is : ZJL216 and min dis = 1.155520550155984
f412a8dfdd851ad3ad757475dc0aa794.jpg
witch is : ZJL207 and min dis = 0.5354013383312322
f41c1b56dab066f2d3fbf850dcea890b.jpg
witch is : ZJL173 and min dis = 0.33816693915476004
f41f2063621c72c1023ec0225d8dcf7d.jpg
witch is : ZJL272 and min dis = 0.10976508019393709
f42145509e1eddc4cfa6ceeef283f4c9.jpg
witch is : ZJL240 and min dis = 1.0434837364403062
f427b39df1ad7f24fdc96a0dcc29fb2f.jpg
witch is : ZJL213 and min dis = 0.3264565787309934
f42a24a3dd6268794340a941d924dab

f60d9896cdcf41e866ad1a20a708f56b.jpg
witch is : ZJL294 and min dis = 1.171589746898456
f613f064e711342e03952300a0f73944.jpg
witch is : ZJL236 and min dis = 0.20379041910525775
f6150174e9b747d30acfa7a8dc26338f.jpg
witch is : ZJL206 and min dis = 0.5141550955247286
f615292dabb0e7429d42d89826173097.jpg
witch is : ZJL236 and min dis = 0.6684648907208571
f6166b87d932a807269007d8e20f6729.jpg
witch is : ZJL253 and min dis = 0.916454590670997
f618425704ee9f659d6c2828733bd351.jpg
witch is : ZJL184 and min dis = 1.1883864421049495
f61b044b81627c3f4c8adbf2c0b06e70.jpg
witch is : ZJL212 and min dis = 0.5573071542042323
f61b2cf660c40198dd7aec7854ceb473.jpg
witch is : ZJL294 and min dis = 0.5576444925027119
f6237722f9efa55f56d0a3979feb3779.jpg
witch is : ZJL164 and min dis = 0.9644408518883946
f62477b0489036165313363b29e621d6.jpg
witch is : ZJL164 and min dis = 0.3753175723289401
f6281ba3940e41a23bdb660bf4d69310.jpg
witch is : ZJL238 and min dis = 0.32371057108430584
f636192164daa8a187b768d121a20e60

f7edfb049f5bf040240b24ffe652519a.jpg
witch is : ZJL210 and min dis = 0.45629715879244054
f7f584ef7784467717aa14cb2db222b1.jpg
witch is : ZJL272 and min dis = 0.4361084894366467
f7f5ec21a5b9283b43572e6c206fb964.jpg
witch is : ZJL166 and min dis = 0.5351648473133975
f7fb77b426bc3a7b30dde8e49cad585b.jpg
witch is : ZJL259 and min dis = 0.5170424267023587
f7ffff6beca29dbc3eb413afc3b19614.jpg
witch is : ZJL216 and min dis = 0.9796580561147397
f80334bd5fbc45e08b09592529e59744.jpg
witch is : ZJL285 and min dis = 0.582154818829584
f80394e468b90b759ded405e3618e920.jpg
witch is : ZJL236 and min dis = 0.24785418596692282
f805c739c23f772c1d492a1016a7ff10.jpg
witch is : ZJL216 and min dis = 1.2119478683867693
f8104db64a6d6014a969fd2cb2a45d98.jpg
witch is : ZJL285 and min dis = 0.3818186695410536
f8132a5a8eed2f6c96c33efba262ec25.jpg
witch is : ZJL210 and min dis = 0.43125044741638924
f814fde6ec675268bd2e7f61bf57a0aa.jpg
witch is : ZJL184 and min dis = 0.43243103064924016
f820a7704c1428627c0784b4ee5ee

fa439dd46ba04e57b0a1ef8a670b8c7e.jpg
witch is : ZJL183 and min dis = 0.22980771609584982
fa45ee1f555dbc14b904b77c310edd06.jpg
witch is : ZJL166 and min dis = 0.467676485425116
fa5513b670facb4fbc9ca62ccccfb6c2.jpg
witch is : ZJL166 and min dis = 0.3421019169256288
fa583a52f13cbdfaedc7ca087f8113e7.jpg
witch is : ZJL257 and min dis = 0.6350541684355453
fa69ee7da3ee9b7117f41e0e04845e9c.jpg
witch is : ZJL294 and min dis = 0.9428945534922402
fa6cd20db74adba6d1b84a619f9d87d4.jpg
witch is : ZJL233 and min dis = 1.1567734291276228
fa6e4554aecef954394f5e39450ed5fd.jpg
witch is : ZJL217 and min dis = 0.8466808120061006
fa6f782f9b71e7a8044fb8c548d8cf35.jpg
witch is : ZJL216 and min dis = 0.8000044320171098
fa734f02159343aa318f201bf84d281c.jpg
witch is : ZJL217 and min dis = 0.65281753206724
fa77bc311eb9b6d9842163191c516077.jpg
witch is : ZJL166 and min dis = 0.665879511361332
fa78359706017cfd65fda6b35ec30598.jpg
witch is : ZJL175 and min dis = 0.34458950518984327
fa840cf5c2a421d97efb5c32665b3974.j

fe9f4b18df2fc6ff8f986d80e057a229.jpg
witch is : ZJL257 and min dis = 0.358149253200173
fea67715739810c2044eb02dc193510c.jpg
witch is : ZJL208 and min dis = 1.1877552843597106
fea881907c72d3b2cc0fac422462b2fa.jpg
witch is : ZJL253 and min dis = 0.5997420662315988
feb6ef566437aa88213337af97c9d93d.jpg
witch is : ZJL236 and min dis = 0.20211714014698695
febeb43c23ac9ba8c7874e8ed627be43.jpg
witch is : ZJL166 and min dis = 0.34297029331670054
fec05f54cdb66919094a781de5c97fc2.jpg
witch is : ZJL166 and min dis = 0.5957146990138941
fecc463a68af5db961140920ad161371.jpg
witch is : ZJL272 and min dis = 0.4731008322794757
fed94097bb14d6d858413a12e0be671b.jpg
witch is : ZJL216 and min dis = 0.9716385616948179
fedd5df1bac9ba5ad5cf8e06a97e56a7.jpg
witch is : ZJL184 and min dis = 0.5641221627779701
fedf1b48b49eeca868d1cff61e1abd5b.jpg
witch is : ZJL166 and min dis = 0.8993259147835532
fee1154fca8844c71198c099228828bf.jpg
witch is : ZJL260 and min dis = 1.371347334001182
fee433ea184b9c91aec0ce5bd7723e86

## 提取部分训练集做测试

In [18]:
#导入一些之后需要的参数
from keras.preprocessing import image

import pandas as pd

import numpy as np

import random

import cv2

data_atten = pd.read_csv(r'test_1.csv')

IDs=[]
for i in range(285):
    IDs.append(data_atten.loc[i][0])
    
data_atten = data_atten.set_index('0')

file = open(r'DatasetB_20180919/train.txt')

data_train = file.readlines()

file.close()

random.shuffle(data_train)

data_retained_test = data_train[:10000]

# file.close()

path = r'DatasetB_20180919/train/'

length = len(data_retained_test)

retained_test_x = np.zeros((length, 64, 64, 3))

retained_test_y = np.zeros((length, 28))
#此处跟随更改

data_atten_res = []

dic={}

count = 0

for ID in IDs:

    key = ID
    print(key)
    
    data_atten_res.append(np.hstack(data_atten.loc[key]))
    
    dic[count] = key

    count = count+1




for i in range(length):

    m,n = data_retained_test[i].split()

    img = image.load_img(path + m)

    
    retained_test_x[i] = image.img_to_array(img)/255

    retained_test_y[i] = data_atten.loc[n]
    


ZJL1
ZJL10
ZJL100
ZJL101
ZJL102
ZJL103
ZJL104
ZJL105
ZJL106
ZJL107
ZJL108
ZJL109
ZJL11
ZJL110
ZJL111
ZJL113
ZJL114
ZJL115
ZJL116
ZJL117
ZJL118
ZJL119
ZJL12
ZJL120
ZJL121
ZJL122
ZJL123
ZJL124
ZJL125
ZJL126
ZJL127
ZJL128
ZJL129
ZJL13
ZJL130
ZJL131
ZJL132
ZJL133
ZJL135
ZJL137
ZJL138
ZJL139
ZJL14
ZJL140
ZJL141
ZJL142
ZJL143
ZJL144
ZJL145
ZJL146
ZJL147
ZJL149
ZJL15
ZJL150
ZJL151
ZJL152
ZJL153
ZJL154
ZJL156
ZJL157
ZJL158
ZJL159
ZJL16
ZJL160
ZJL161
ZJL162
ZJL163
ZJL164
ZJL165
ZJL166
ZJL167
ZJL168
ZJL169
ZJL170
ZJL171
ZJL172
ZJL173
ZJL174
ZJL175
ZJL176
ZJL177
ZJL178
ZJL179
ZJL18
ZJL180
ZJL181
ZJL182
ZJL183
ZJL184
ZJL185
ZJL186
ZJL187
ZJL188
ZJL189
ZJL19
ZJL190
ZJL191
ZJL192
ZJL193
ZJL194
ZJL195
ZJL196
ZJL197
ZJL198
ZJL199
ZJL2
ZJL200
ZJL201
ZJL202
ZJL203
ZJL204
ZJL205
ZJL206
ZJL207
ZJL208
ZJL209
ZJL21
ZJL210
ZJL211
ZJL212
ZJL213
ZJL214
ZJL215
ZJL216
ZJL217
ZJL218
ZJL219
ZJL22
ZJL220
ZJL221
ZJL222
ZJL223
ZJL224
ZJL225
ZJL226
ZJL227
ZJL228
ZJL229
ZJL23
ZJL230
ZJL231
ZJL232
ZJL233
ZJL234
ZJL235
Z

In [20]:
res = model.predict(retained_test_x)

In [21]:
#将预测输出进行处理，组合相对应的
vec_result = []#np.zores((len(res[0]), 18))

for i in range(len(res[0])):

    vec_result.append(np.hstack((res[0][i], res[1][i], res[2][i],res[3][i])))

length = len(vec_result)



In [22]:
# file  = open("retaines_test.txt",'a+')
accury = 0 
for i in range(length):
    dis_list=[]
    m,n = data_retained_test[i].split()
    for j in range(len(data_atten_res)):

        vec1 = vec_result[i]
        
        vec2 = data_atten_res[j]
        

        dist = np.linalg.norm(vec1 - vec2)

        dis_list.append(dist)
        
    print(m)
    print("witch is : "+ dic[dis_list.index(min(dis_list))] + " and min dis = " + str(min(dis_list)))
    if n == dic[dis_list.index(min(dis_list))]:
        accury += 1
        print('true')
#     file.write(m+'\t'+dic[dis_list.index(min(dis_list))]+'\n')
# file.close()

97fbfe001835ea7c2918f05c17eda3f5.jpeg
witch is : ZJL61 and min dis = 0.045200263500174136
true
2d24dd5724ac70bd458a5f24a67875e1.jpeg
witch is : ZJL40 and min dis = 0.059570444403087584
true
9e70b9566af2069e4e5f7eea6df5cc27.jpeg
witch is : ZJL141 and min dis = 0.08896081594529974
true
3fa58c340823007c579a511b73e454e2.jpeg
witch is : ZJL245 and min dis = 0.0760965102252734
f1761da6a478a435abe98e0e71891311.jpeg
witch is : ZJL127 and min dis = 0.008703596688216839
true
2aa2dca03ff77f17b18a8851a3ed52ff.jpeg
witch is : ZJL110 and min dis = 0.10656045603278058
true
6a969a8df2e8efddb9daec223013f4af.jpeg
witch is : ZJL159 and min dis = 0.07143053677610436
true
74b0f548e92f29d263341ba2373b5e7a.jpeg
witch is : ZJL12 and min dis = 0.18771963438825384
true
48cfc7eb6ab050069cd3eee571c6a7c2.jpeg
witch is : ZJL44 and min dis = 0.07343980074038353
true
072e908c480cfd24bf87a3c017ca641e.jpeg
witch is : ZJL48 and min dis = 0.12125302969796171
true
404a6d459d8d3c2c15307c71dcb8781a.jpeg
witch is : ZJL130 an

abc651fbe6a18a70611bcb0cc8475d0d.jpeg
witch is : ZJL265 and min dis = 0.018125331374248377
true
dc4ab2f0278d4568c3c70d64093ff39d.jpeg
witch is : ZJL111 and min dis = 0.1402456691180783
true
baaa3c25727a01ce94fc6053c50470c6.jpeg
witch is : ZJL144 and min dis = 0.2986036357881973
610bb49d91d26986f7a2edb95c050a40.jpeg
witch is : ZJL244 and min dis = 0.05325483515336336
true
fd5c8cdecbcd82af64da3d28479dd34d.jpeg
witch is : ZJL127 and min dis = 0.01999203518668028
true
1414b13c4ec12aff8b6e28d642c3e3ad.jpeg
witch is : ZJL15 and min dis = 0.16051976324330405
true
27d31c90c50ccc7cd2f25168edd0527d.jpeg
witch is : ZJL245 and min dis = 0.06578214964989873
a5de2816774dd488cd6e94f103739a62.jpeg
witch is : ZJL116 and min dis = 0.0192679112762844
true
b1d79775e8abd771bac7e124212044f5.jpeg
witch is : ZJL169 and min dis = 0.09913369348529107
6e37910b8ca962f5e6d92fe89222e84b.jpeg
witch is : ZJL247 and min dis = 0.14435053187620986
true
32c0d0083e4c5512056bfa776f517b26.jpeg
witch is : ZJL117 and min dis 

a077fe7efe1c92d89a3fa3734ad883eb.jpeg
witch is : ZJL68 and min dis = 0.10204606965410394
true
f81cc90265e9d8f0506abbbe56d78f52.jpeg
witch is : ZJL133 and min dis = 0.06037487251421291
true
cc3e341bb9992e8e3cfec56134df95f9.jpeg
witch is : ZJL158 and min dis = 0.09060748817194524
true
83d65ac4396865d648c54ab0f28ee48b.jpeg
witch is : ZJL48 and min dis = 0.11745529131018453
true
334944840563c396e1ffeeb66572466a.jpeg
witch is : ZJL13 and min dis = 0.11319545626503996
true
8a38a646437f48f0607b24aef2342817.jpeg
witch is : ZJL40 and min dis = 0.023162136987782533
5939031937d7379a070713f3a4a209d2.jpeg
witch is : ZJL11 and min dis = 0.012759385692940857
9b44b9dbe46dca32f75ae86be4d23f79.jpeg
witch is : ZJL139 and min dis = 0.11990943581454133
true
5de8f7f8a59316783c41c79783f68542.jpeg
witch is : ZJL268 and min dis = 0.7858998673685634
8163416a81bd2e9a355daa65cd6f025b.jpeg
witch is : ZJL211 and min dis = 0.023764207443458793
5eef2b9e8e8102b2e3e6d55d1750848e.jpeg
witch is : ZJL126 and min dis = 0.1

d8ee88262dbf17c3b706d4932552ba60.jpeg
witch is : ZJL119 and min dis = 0.027186832842408568
true
a9558219b85d011f3d612c2291299886.jpeg
witch is : ZJL100 and min dis = 0.07091117456525797
true
70a16176bd3dbfb0cd748d0b2274ac39.jpeg
witch is : ZJL130 and min dis = 0.14242941577658194
true
3fdc90af0a63c1cd11c7f656546bff9a.jpeg
witch is : ZJL140 and min dis = 0.15908104071319143
true
82d911675a60d6e8d0cf090b89baa60b.jpeg
witch is : ZJL47 and min dis = 0.08189873816333496
true
2cde213b7766a517a30be63203e9468d.jpeg
witch is : ZJL110 and min dis = 0.10175939883078704
true
b0031c5b26fc08383c0ed30521c3bf30.jpeg
witch is : ZJL245 and min dis = 0.07025498823282132
0504768d386a3d22e946673915145eac.jpeg
witch is : ZJL191 and min dis = 0.107645992140922
2ece314ddec326e829aab38f7918e489.jpeg
witch is : ZJL55 and min dis = 0.1468089053623755
true
b6d58d50af18bf96784646ca18bc329b.jpeg
witch is : ZJL128 and min dis = 0.24331258318678048
true
53cb35c1d8e206f8789ce28931447254.jpeg
witch is : ZJL126 and min 

0906ab176baccff82e0158b08f8b9559.jpeg
witch is : ZJL18 and min dis = 0.06605031885201014
df5c12dcefffe49e5bb9b31b543d892c.jpeg
witch is : ZJL71 and min dis = 0.07433072757018999
true
67edd81951c00499261520c81ef176f2.jpeg
witch is : ZJL29 and min dis = 0.12733694207700394
true
191864503d65d2d31ddea1b56290d9b1.jpeg
witch is : ZJL263 and min dis = 0.13932799773117202
true
2a9cf175f2560df0313d21e0c82f602a.jpeg
witch is : ZJL51 and min dis = 0.03130364081977375
d518b328cae3cb9a98cf769114355258.jpeg
witch is : ZJL264 and min dis = 0.14872115677464381
true
1591265ebfc60f4b2fda835ccd49fee0.jpeg
witch is : ZJL89 and min dis = 0.17741296623932973
59409761f6c550c1c50a555b8e5b3f84.jpeg
witch is : ZJL54 and min dis = 0.11225121958263648
true
d900b24520df79796787a70e970620f2.jpeg
witch is : ZJL143 and min dis = 0.09521317003913375
true
088c860829dd74076654f4fdda012c2c.jpeg
witch is : ZJL127 and min dis = 0.051071820549133805
true
cf70053c0d902926be4fcb8c6d664791.jpeg
witch is : ZJL11 and min dis = 0

...

460fca533a136b820b9d673aee2bd3e4.jpeg
witch is : ZJL35 and min dis = 0.04572605623008896
true
409aac7ada543d0fe325c65848845435.jpeg
witch is : ZJL147 and min dis = 0.11005117644884847
true
291bf317ebb5b533c5a4e552e0bbe6c8.jpeg
witch is : ZJL10 and min dis = 0.09075203472823834
true
24bd2d940edd9e61bc7ea7517d71fb9a.jpeg
witch is : ZJL65 and min dis = 0.19202526843713652
true
50501b4bdf7f7efe34b3b66361a25ba4.jpeg
witch is : ZJL126 and min dis = 0.09735275510757943
true
81f6894ab3475a28230acd46c963d3f0.jpeg
witch is : ZJL70 and min dis = 0.10289526158429317
6a6511f8a0a2f64c923babecf28db436.jpeg
witch is : ZJL107 and min dis = 0.09014561657018538
true
bc274ba9ba10e0a2193134dd1cbb3d24.jpeg
witch is : ZJL100 and min dis = 0.1400543382886534
true
8f1293ab19f832e865b7e70c8b97d880.jpeg
witch is : ZJL85 and min dis = 0.13334178954518036
true
6b79c30f7fd05922a27d8203bbd8c833.jpeg
witch is : ZJL11 and min dis = 0.009064301737138288
6a92b6ac67fca98ae93d09b6257dc9ed.jpeg
witch is : ZJL68 and min dis

7fc0aced01afbd91af3bbb58ae76b9f0.jpeg
witch is : ZJL130 and min dis = 0.12648268118252456
true
782f4dd22bfe6978b9495d4070bf3240.jpeg
witch is : ZJL138 and min dis = 0.02471474425519647
true
f0ff237000721e467d8c31c7d4f0e8f3.jpeg
witch is : ZJL264 and min dis = 0.1391009384932242
true
9b07eb61293aebf89a2aff26e615f841.jpeg
witch is : ZJL138 and min dis = 0.05793791094334399
true
ad949b5034437612c5e85be56236de39.jpeg
witch is : ZJL119 and min dis = 0.008971820029465451
true
b33d9a1241bf0fcd39546d2e19b8acaa.jpeg
witch is : ZJL115 and min dis = 0.0731618722358448
true
243898f2887b845eace1cda313ca00d3.jpeg
witch is : ZJL140 and min dis = 0.23825138942692758
6c3f9a61e3e2aa134a5b7db1813e12b1.jpeg
witch is : ZJL265 and min dis = 0.008076765030175678
true
a349e95c030b2c7d5107d8383e3c34d3.jpeg
witch is : ZJL57 and min dis = 0.5289368803929254
3a8e2dd08c04b7c66e6c61c7d9195116.jpeg
witch is : ZJL120 and min dis = 0.05310670552291688
7ea988503967b3dedc4654685b531113.jpeg
witch is : ZJL263 and min dis

48d4536e126d5f03e9777ddd064ca2df.jpeg
witch is : ZJL51 and min dis = 0.01697985032763095
true
f922c178d02504ed44f63c91660eb89e.jpeg
witch is : ZJL2 and min dis = 0.06374496934498602
true
f5834e9546463389c2f643461f06743d.jpeg
witch is : ZJL68 and min dis = 0.08860982594802938
true
57f72d227baafc723aff55778c7e1c90.jpeg
witch is : ZJL147 and min dis = 0.38385529309840893
d19f31468b44ba2f376969306a820321.jpeg
witch is : ZJL117 and min dis = 0.07738396994764274
true
cafa1b3addad3992c3c64e4066f0c7b5.jpeg
witch is : ZJL275 and min dis = 0.34682535501038125
ddbe5dd0e462b47c2927c4aba4ba08e8.jpeg
witch is : ZJL107 and min dis = 0.11001422995822802
true
68f25d2f2b91b67ec49c0f2ed02ee539.jpeg
witch is : ZJL151 and min dis = 0.08917232199515747
true
364569bc306c16426c0f60e3b652c882.jpeg
witch is : ZJL70 and min dis = 0.047102560215038135
true
0c2f142a0212eae18e92991565ea1e72.jpeg
witch is : ZJL87 and min dis = 0.11255795224800967
true
8c722e76105e92e5399d487ba5c8aef9.jpeg
witch is : ZJL119 and min d

079af6ea24ccdbbc5ce173dcc0491567.jpeg
witch is : ZJL147 and min dis = 0.2012528965897935
true
263732af668818b81387f4ae56468077.jpeg
witch is : ZJL93 and min dis = 0.1284162980325164
true
94032a6c0d7d2c728f3dde5772355222.jpeg
witch is : ZJL38 and min dis = 0.09187440679872506
5a1a222d5a78ba97c8cbc59bd2fec89d.jpeg
witch is : ZJL146 and min dis = 0.06782976433718284
true
4df8417f49f51124412a6ca7b37434f1.jpeg
witch is : ZJL11 and min dis = 0.005618052508994164
e40189eb0ae3047314000af8a1267d20.jpeg
witch is : ZJL70 and min dis = 0.07229783853386579
7988cedb9be208438a504959df33f818.jpeg
witch is : ZJL156 and min dis = 0.1034074024586981
true
6e5f297d02242f1d9cc30a0631b0da71.jpeg
witch is : ZJL24 and min dis = 0.0124046179599857
true
b3b86bd0035852659ea8b6032dda5fd6.jpeg
witch is : ZJL88 and min dis = 0.16539760097117187
true
8498db88754c7cd6395fa4685233c953.jpeg
witch is : ZJL4 and min dis = 0.04143827368551937
true
e78eedbd5f88948cf469a9dea7f197a4.jpeg
witch is : ZJL60 and min dis = 0.07680

d3bb86757fd48c551f08f7c204524f78.jpeg
witch is : ZJL216 and min dis = 0.7385242607605773
36db6ce30b68b9845af97ae8004cae65.jpeg
witch is : ZJL107 and min dis = 0.1024425255246761
true
821c5ad51599c07f782ecee26fdb89b0.jpeg
witch is : ZJL129 and min dis = 0.07827889067939799
true
0c72882c59af1afc7d086e8a438f51d5.jpeg
witch is : ZJL263 and min dis = 0.13313428770429261
true
2cc73a602844c4df1ea2d4d95dafd776.jpeg
witch is : ZJL113 and min dis = 0.12612527058650885
true
5cf49970574288529f0e409db394c82f.jpeg
witch is : ZJL48 and min dis = 0.11932798575507361
true
4e1954759be7dc067b3354d36af52fc3.jpeg
witch is : ZJL70 and min dis = 0.06637388137129542
c50bd1fffea91f5bdcd0f0b542e8ef2b.jpeg
witch is : ZJL169 and min dis = 0.2867908548391006
34832ed8434af40f4c1976443176df66.jpeg
witch is : ZJL67 and min dis = 0.1527690777509389
true
92b8dbaa11822c28b560ce4b99d95094.jpeg
witch is : ZJL244 and min dis = 0.058196950770765365
true
0cd06b70f8f5fb4e9956c87e9111a77b.jpeg
witch is : ZJL21 and min dis = 0.

In [23]:
accury/length

0.7657